In [ ]:
import torch
import os
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from torchvision.models.detection.faster_rcnn import fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torchvision.transforms as T

model = fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)
# Modify the classifier to fit the number of classes
num_classes = 9  # Your number of classes + background
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)


model.load_state_dict(torch.load('model/model_weights.pth'))

In [ ]:
# Splitting the path for clarity and correctness
base_path = '../Tower/data'
sub_folder = 'P10/H2-DOWN/img'
file_name = 'P10STR_104.17_106.46_019.55_021.83_H2_DOWN_038_Q0224.jpeg'  # Add file extension if needed, e.g., '.jpg'

# Construct the path using os.path.join
file_path = os.path.join(base_path, sub_folder, file_name)

# Load the image
img = Image.open(file_path).convert('RGB')

# Convert the PIL Image to a PyTorch Tensor
# Instead of manually converting and normalizing, let's use torchvision transforms
transform = T.Compose([
    T.ToTensor(),  # Converts to Tensor and scales to [0, 1])  # Assuming your model expects 224x224 images
])

img_tensor = transform(img)

# Add a batch dimension since PyTorch models expect batches
img_tensor = img_tensor.unsqueeze(0)

# Determine the device dynamically
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
img_tensor = img_tensor.to(device)  # Move tensor to the appropriate device

# Ensure the model is on the same device and set it to evaluation mode
model = model.to(device)
model.eval()

# Perform the prediction
with torch.no_grad():
    prediction = model(img_tensor)

# Process the prediction output as needed
print(prediction)

# Display the image (if you're in an environment that supports this, like Jupyter)
img.show()

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import torchvision.transforms.functional as TF

im = TF.to_pil_image(img_tensor.squeeze().cpu())
draw = ImageDraw.Draw(im)

for index, box in enumerate(prediction[0]['boxes'].cpu().numpy()):
    if prediction[0]['scores'][index] > 0.24:
        draw.rectangle(box, width=5, outline="red")
        text = str(prediction[0]['labels'][index].item())
        text = text + ' score: ' + str(round(prediction[0]['scores'][index].item(),2))
        font = ImageFont.truetype("arial.ttf", size=40)
        text_position = (box[0], box[3])
        draw.text(text_position, text, fill="red", font=font)

im